In [98]:
import numpy as np
import random as ran
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras.utils.np_utils import to_categorical

# 1) Create ToyModel

In [85]:
#************** CONSTANTS ***************
      
pi=np.pi
mass=0.0001 

#Definisce dei file con label 10,12,13,11. A fine del vecchio programma venivano salvati questi 4 file
#Noi siamo interessati al 10

#open(10,file='toymodel.dat',status='unknown')
#open(12,file='toymodel1-rand.dat',status='unknown')
#open(13,file='toymodel2-rand.dat',status='unknown')
#open(11,file='toymodel_prof.dat',status='unknown')

x8=1.000000000000
Np=500000
Ldisc=600.
print('Size  disk=%f' %Ldisc)

iota=0. 
print('iota=%f' %iota)

L1=1.5*Ldisc
L2=1.*Ldisc
L3=Ldisc/10.
print('L1,L2,L3=%f%f%f' %(L1,L2,L3))

#print('L1,L2,L3 [semi axis]=',L1,L2,L3) #---> non capisco
      
print('Np= %i' %Np)
      
      
#*************************************************
x=np.zeros((Np))
y=np.zeros((Np))
z=np.zeros((Np))
r=np.zeros((Np))
vxRd=np.zeros((Np))
vyRd=np.zeros((Np))
vzRd=np.zeros((Np))
vxRt=np.zeros((Np))
vyRt=np.zeros((Np))
vzRt=np.zeros((Np))
vx=np.zeros((Np))
vy=np.zeros((Np))
vz=np.zeros((Np))
vtot = np.zeros((Np))

x[0]=0.
y[0]=0.
z[0]=0.
       
#**************** disk ***************************

#j=0                                                 A COSA SERVE QUESTO j?

#Per ogni punto
for i in range(1,Np):
    
    #x1, y1 e z1 devono rispettare delle condizioni; creo un ciclo while che si ferma quando vengono rispettate
    #Per entrare nel while setto le 3 variabili (ed r2) in modo tale da entrare nel ciclo
    x1 = L1 + 1
    y1 = L2 + 1
    z1 = L3 + 1
    r2 = 2.
    
    while(((x1 < -L1) or (x1 > L1)) and ((y1 < -L2) or (y1 > L2)) and ((z1 < -L3) or (z1 > L3)) and (r2 > 1)):
        x1=2.*L1*(ran.uniform(0,1)-0.5)
        y1=2.*L2*(ran.uniform(0,1)-0.5)
        z1=2.*L3*(ran.uniform(0,1)-0.5)
        #Non inserisco rz perché non serve per rispettare le 4 condizioni 
        rx=x1-x[0]
        ry=y1-y[0]

        r2 = np.sqrt(rx*rx/(L1)**2.+ry*ry/(L2)**2)

    #Uscito dal while faccio tutte le assegnazioni
    x[i]=x1
    y[i]=y1
    z[i]=z1
    
    rx=x[i]-x[0]
    ry=y[i]-y[0]
    rz=z[i]-z[0]
    
    r[i] = np.sqrt(rx*rx+ry*ry+rz*rz)
    alpha = 180./pi*np.arctan(np.abs(ry)/np.abs(rx))
    
    
    if(r[i] <= 200):
        vradial = 0.
        vrotation = 100.

    else: #---> PARTE DA MODIFICARE
        vradial = 0.4*r[i]
        vrotation = 000./(r[i]/200.)

    vxRd[i] = abs(vradial)*np.abs(rx)/r[i] #radial component 
    vyRd[i] = abs(vradial)*np.abs(ry)/r[i]
    vzRd[i] = abs(vradial)*np.abs(rz)/r[i]

    if(rx < 0):
        vxRd[i] = -vxRd[i]
    if(ry < 0):
        vyRd[i] = -vyRd[i]
    if(rz < 0):
        vzRd[i] = -vzRd[i]


#       vRad=sqrt(vxRd(i)*vxRd(i)+vyRd(i)*vyRd(i)+vzRd(i)*vzRd(i))


#       vxRt(i)=+(ry)*vrotation/r(i) !clockwise --> anticlockwise invert signs
#       vyRt(i)=-(rx)*vrotation/r(i)
#       vzRt(i)=0.

    vxRt[i] = -(ry)*vrotation/r[i] #anticlockwise --> clockwise invert signs
    vyRt[i] = +(rx)*vrotation/r[i]
    vzRt[i] = 0.

    vRot = np.sqrt(vxRt[i]*vxRt[i]+vyRt[i]*vyRt[i]+vzRt[i]*vzRt[i])

    vx[i] = vxRt[i]+vxRd[i]
    vy[i] = vyRt[i]+vyRd[i]
    vz[i] = vzRt[i]+vzRd[i]

    vtot[i] = np.sqrt((vx[i]**2.+(vy[i]**2.+(vz[i]**2.))))

    #write(10,*)x(i),y(i),z(i),vx(i),vy(i),vz(i),x8 ---> Questo è il file che poi, prima di questo codice,
                                                        #assegnavamo al codice python che proietta sulla
                                                        #linea di vista

Size  disk=600.000000
iota=0.000000
L1,L2,L3=900.000000600.00000060.000000
Np= 500000


# Al posto di 1): fare rete neurale che genera vradiale (vR) e vcircolare (vT) + l'angolo j, tutti e 3 da dare in input al programma in 2) che generera' la vlinea di vista

In [99]:
#Vreal = np.loadtxt('file.txt')
Vreal = np.zeros((10000))

In [102]:
input_layer = keras.Input(shape=(Vreal.shape[0], ))
encoder_layer_1 = keras.layers.Dense(500, activation="relu")(input_layer)
encoder_layer_2 = keras.layers.Dropout(.2, input_shape=(400,))(encoder_layer_1)
encoder_layer_3 = keras.layers.Dense(300, activation="relu")(encoder_layer_2)
encoder_layer_4 = keras.layers.Dense(100, activation="relu")(encoder_layer_3)
output_layer = keras.layers.Dense(Vreal.shape[0]*2+1, activation="sigmoid")(encoder_layer_4)
model = keras.Model(inputs=input_layer, outputs=output_layer)
model.summary()
#output_layer1 = keras.layers.Dense(Vreal.shape[0], activation="sigmoid")(encoder_layer_4)
#output_layer2 = keras.layers.Dense(Vreal.shape[0], activation="sigmoid")(encoder_layer_4)
#output_layer3 = keras.layers.Dense(1, activation="sigmoid")(encoder_layer_4)
#model = keras.Model(inputs=input_layer, outputs=(output_layer1,output_layer2,output_layer3))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 10000)]           0         
_________________________________________________________________
dense_12 (Dense)             (None, 500)               5000500   
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 300)               150300    
_________________________________________________________________
dense_14 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_15 (Dense)             (None, 20001)             2020101   
Total params: 7,201,001
Trainable params: 7,201,001
Non-trainable params: 0
_________________________________________________

In [ ]:
# Optimizer,loss, metrics and compile the model
# learning rate
LR_ST=1e-3
# Optimizer:
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LR_ST)
# Loss function:
LOSS = tf.keras.losses.BinaryCrossentropy(from_logits=False)
METRIC = [tf.keras.metrics.BinaryCrossentropy()]
# Model
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRIC)

# 2) Proiezione lungo la linea di vista (è già file python, quindi dovrebbe essere semplice)

In [ ]:
I = Iinput/180.*np.pi
J = Jinput/180.*np.pi
Vloss = vT*np.sin(I)*np.cos(J)+vR*(np.sin(I)*np.sin(J))